In [180]:
import sys
import math
import networkx as nx
from collections import OrderedDict


reads="/panfs/qcb-panasas/rdagnew/HG00514/pbmm_combined/all_3/chr6:160609100-160648600/ccs/reads.fasta"
cov=26

psv="/panfs/qcb-panasas/rdagnew/HG00514/pbmm_combined/all_3/chr6:160609100-160648600/ccs/kmer_filter/filtered.orig_counts.nucfreq"
k=32
out="/home/cmb-16/mjc/rdagnew/summerproj/kmer_filter/psv_graph/psv.ccs_filt.gml"



PSV = OrderedDict()
MG = nx.MultiDiGraph()
#MG.add_node("s")

In [181]:
def storePSVfilt(PSV,PSVlist,MG,k):
    l=0
    _refnode = ""
    _node = ""
    info=""
    pre=""
    for n in PSVlist:
        if n[0]=="#":
            continue  
       # print(n)
        n = n.rstrip()
        n = n.split()
      #  print(n)
        info = n[9]
        kmer = info.split(";")[4]
        pre = kmer.split("=")[1].split(",")
        _refnode = pre[0]
        _node = pre[1]
        PSV[_node] = []
        kmer = ""
#        assert k == len(line)
        for i in range(k):
            kmer = _node[i:i+k]
            PSV[_node].append(kmer)
            MG.add_node(kmer , pos=n[1], ref=_refnode[k-1], alt=_node[k-1] ,of=i)
            l=l+1
    print("stored filt PSV list:",str(l))

In [182]:
        
#set but need to keep track of multiplicity
PSVlist=[]
with open(psv) as p:
    PSVlist=p.readlines()
#print(PSVlist)
print(MG.number_of_nodes())
storePSVfilt(PSV, PSVlist, MG,k)
print(MG.number_of_nodes())
#print(PSV)

read_name = ""
offset = 0
starter=0
ender=0
un_read=0
read_stream = open(reads)

read_count = 0
node_multi = {}
for key in PSV.keys():
    for kmer in PSV[key]:
        node_multi[kmer]=0


0
stored filt PSV list: 640
544


In [183]:
for key in PSV.keys():
    for kmer in PSV[key]:
        node_multi[kmer]=0

for line in read_stream:
    if line[0] == ">":
        line = line.lstrip()
        read_name = "/".join(line.split("/")[:2])
        continue
    read = line.rstrip()
    read_len = len(read)
    first_match = 1
    read_count = read_count+1

    sentinel = "_".join(["t", read_name ])
    _starter = "_".join(["s", read_name ])
    
    
    for i in range((read_len - k)+ 1):
        kmer = read[i:i+k]


        for key in PSV.keys():
            current_off=-1
            if kmer in PSV[key]:
                offset = PSV[key].index(kmer)
            #    print(kmer)

                if MG.has_node(kmer):
                    node_multi[kmer] = node_multi[kmer] + 1
                    MG.add_node(kmer, mult=node_multi[kmer])
                else:
                    node_multi[kmer] = 1
                    MG.add_node(kmer, mult=1)


                if first_match and i == read_len - k:
                    MG.add_node(_starter)
                 #   MG.add_edge("s", _starter)
                    MG.add_node(sentinel)
                    
                    MG.add_edge(_starter, kmer, rn=read_name, of=offset, ii=i, rl= read_len)
                    MG.add_edge(kmer, sentinel, rn=read_name, ii=i, rl= read_len)
                    continue

                if first_match:
                    MG.add_node(_starter)
                   # MG.add_edge("s", _starter)
                    MG.add_edge(_starter, kmer, rn=read_name, of=offset, ii=i,rl= read_len)
                    current_match = kmer
                    current_off = offset
                    first_match = 0
                    starter = starter +1 
                    continue

                if  i == read_len - k:
                    MG.add_node(sentinel)
                    MG.add_edge(kmer, sentinel, rn=read_name, ii=i,rl= read_len)
                    ender = ender +1 
                    continue
                                                
                MG.add_edge(current_match, kmer, rn=read_name, ii=i, iof = current_off,rl= read_len, of=offset)
                current_match = kmer
                current_off = offset
                
            elif not first_match and i == read_len - k:
                MG.add_node(sentinel)
                MG.add_edge(current_match, sentinel, rn=read_name, ii=i , iof = current_off,rl= read_len)
                ender = ender +1 

    if first_match:
        un_read = un_read+1            
    if read_count%10 == 0:
        #print(read_count)
        print(MG.number_of_edges())
    if read_count%100 == 0:
        print(read_count)



191
991
1177
1177
1177
1177
1197
1246
1295
1382
100
1498
1592
1681
1681
1824
1824
1873
1943
1978
1978
200
1978
2027
2061
2159
2326
2375
2671
2842
2971
3038
300
3552
4785
5644
6690


In [184]:

print(MG.number_of_edges())
print(MG.number_of_nodes())

6690
714


In [185]:
print("reads: ",str(read_count),"discard reads: ",str(un_read),"\nretain nodes: ",str(MG.number_of_nodes()),"discard nodes: ",str(un_node))
print("source: ",str(starter),"\nsink: ",str(ender))

reads:  345 discard reads:  260 
retain nodes:  714 discard nodes:  103
source:  85 
sink:  1445


In [186]:
def contract(g):
    is_chain = [node for node in g.nodes() if g.in_degree(node)==g.out_degree(node)]
    chains = g.subgraph(is_chain)
   # nx.components.connected_component_subgraphs(chains)
    #print(chains.nodes())
    return is_chain 
  

In [187]:
pre=contract(MG)
print(len(pre))
print(MG.number_of_nodes())

532
714


In [190]:
ini=1
ct=1
deg=-1
master=""
prev=""

for node in MG.nodes():#.nodes():
    if ini:
        master=node
        prev=node
#        deg= MG.out_degree(node)
        ini=0
    elif not ini:
        if prev[1:] == node[:-2]:
            MG = nx.contracted_nodes(MG,master,node)
            prev=node
        else:
            ini=1
        
    
    
    
    

In [150]:
ini=1
ct=1
deg=-1
master=""
merged=[]

for node in pre:#.nodes():
    if ini:
        ct=ct+1
        master=node 
        deg= MG.out_degree(node)
        ini=0
    if not ini and ct>1 and deg== MG.in_degree(node):
        merged.append(node)    
    elif not ini and ct>1 and deg != MG.in_degree(node):
        for n in merged:
            MG = nx.contracted_nodes(MG,master,n)
        ini=1
        ct=1
        deg=-1
        master=""
        merged=[]
    
    
    
    
    

In [191]:
print(MG.number_of_nodes())

714


In [167]:
MG.add_nodes_from(PSVlist)

In [173]:
print(len(PSVlist))
print(MG.number_of_nodes())

70
1916


In [177]:
i=0
j=0
for key in PSV.keys():
    node=key
    i=i+1
    print("i",i)
    for kmer in PSV[key]:
        j=j+1
        print("j",j)
        MG = nx.contracted_nodes(MG,node,kmer)
        

i 1
j 1


KeyError: 'ATTCTTCTATTCTTCTTATACCTAATTCTTTT'

In [192]:
un_node=0
rem=[]
for node in MG.nodes():
    if MG.degree(node) == 0:
        rem.append(node)
        un_node = un_node+1
MG.remove_nodes_from(rem)

In [193]:
nx.write_gml(MG,out)